
# Time Complexity Analysis for The $S^2$ Data Generation

In this section, we provide a detailed analysis and proof of the time complexity of the $S^2$ data generation mechanism. Our theoretical analysis shows that the time complexity of data generation is proportional to the length $L$ of the time series. We will then verify the specific time required for data generation using multiple sets of different lengths to validate our theoretical analysis.

We define the specific symbol explanation and its  as follows. Then we use the divide-and-conquer approach to analyze the complexity of our $S^2$ data generation mechanism.

+-----------+----------------------------------------------------------------------+
| symbol    | explanation                                                          |
+===========+======================================================================+
| $L$ | The length of time series                                            |
+-----------+----------------------------------------------------------------------+
| $M$ | Number of input channels                                             |
+-----------+----------------------------------------------------------------------+
| $N$ | Number of output channels                                            |
+-----------+----------------------------------------------------------------------+
| $k$ | Total number of mixed distributions used                             |
+-----------+----------------------------------------------------------------------+
| $p$ | Autoregressive model order in ARMA model                             |
+-----------+----------------------------------------------------------------------+
| $q$ | Moving average model order in ARMA model                             |
+-----------+----------------------------------------------------------------------+
| $P$ | Probability of choosing a sampling method                            |
+-----------+----------------------------------------------------------------------+
| $b$ | Number of binary operators used to construct symbolic expressions    |
+-----------+----------------------------------------------------------------------+
| $u$ | The number of unary operators used to construct symbolic expressions |
+-----------+----------------------------------------------------------------------+
 
1. **Symbolic Expression Generation**: We construct symbolic expressions using a tree structure as a medium. When we have $b$ binary operators, we further insert $(b + 1)$ leaf nodes (the process from (a) to (b) in **Figure 3** in our paper). Therefore, after inserting $u$ unary operators (**Figure 3** (c)), the total number of nodes in the tree is $n = 2b + u + 1$. Because there are many ways to construct a tree, we consider the time complexity of constructing a balanced tree. Therefore, for $N$ symbols constructed, the specific complexity of this process is:

\begin{align}O(N\times n\mathrm{log}n)\end{align}


2. **Sampling series generation**: When we want to generate a sampling time series with $M$ channels, each channel has a probability of :math`:`P` to be sampled using a mixture distribution and a probability of $(1-P)$ to be sampled using an ARMA model. When the sampling length of the series is $L$, the complexity of generating $k$ mixture distribution and ARMA ($p$, $q$) series is $O(kL)$ and $O(L(p+q))$. Therefore, the time complexity of this process can be quantified as:

\begin{align}O \left ( ML \times [Pk + (1-P)(p+q)] \right )\end{align}


3. **Sampling through symbolic expressions and series**: We simplify the specific operational details of this process and only consider the time complexity of operations with variables. For a series of length L, we have $N$ symbolic expressions to be sampled, and each symbol has an average of $\frac{M+1}{2}$ variables (Each symbolic expression may contain any number of variables from 1 to M, so here we take $\frac{M+1}{2}=\frac{(1+2+\cdots+M)}{M}$ as the average probability). Then the process can be quantified as:

\begin{align}O(N \cdot \frac{M+1}{2}\cdot L)\end{align}

To sum up, since other variables that affect the $S^2$ sampling process are usually small, it can be intuitively understood that the time complexity of the entire sampling process is proportional to the length $L$.


In [ ]:
import time
import numpy as np
from tqdm import tqdm

from S2Generator import Generator, SymbolParams

In the following experiment, we generate time series data with a length interval of 16 and calculate the specific time required.



In [ ]:
# Create the generator instance
generator = Generator(symbol_params=SymbolParams(max_trials=16))

# The number of generated data
try_count = 1

length_array = np.arange(16, 528, 16)

time_array = np.zeros_like(length_array)

# Variables with different time lengths
for index, n_points in enumerate(length_array):
    start = time.time()
    for seed in tqdm(range(try_count)):
        # Create the random number generator
        rng = np.random.RandomState(seed)

        # Start generating data
        generator.run(
            rng=rng, input_dimension=1, output_dimension=1, n_inputs_points=n_points
        )

    # Record the time required for this length
    end = time.time()
    time_array[index] = end - start

    # Print status information
    print(f"Generate Length: {n_points}, Time: {end - start}!")

From the above experimental results, we can generally see that the time required for data generation is generally proportional to the length of the time series.
However, consider that in many cases, when we construct the symbolic expression $f(\cdot)$ and input the stimulus time series $X$ into the system to obtain the corresponding $Y=f(X)$, the values ​​of the stimulus time series may fall outside the domain of the symbolic expression $f(\cdot)$, resulting in sampling failure. This phenomenon will affect the sampling time.



In [ ]:
from matplotlib import pyplot as plt

fig, ax = plt.subplots(figsize=(8, 3), dpi=180)

ax.plot(
    length_array,
    time_array,
    color="royalblue",
    marker="o",
    markersize=5,
    markerfacecolor="white",
)
ax.grid("--", color="gray", alpha=0.4)
ax.set_xlabel("The length of the generated time series", fontsize=12)
ax.set_ylabel("The Time Complexity Analysis", fontsize=12)

# %%